In [1]:
import seaborn as sns
import metapack as mp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display 

import datetime
from cityiq import Config, CityIq
from cityiq.api import Asset, Location
from cityiq.scrape import EventScraper

%matplotlib inline
sns.set_context('notebook')
mp.jupyter.init()


In [2]:
#pkg = mp.jupyter.open_package()
pkg = mp.jupyter.open_source_package()
pkg

In [3]:

tz = datetime.datetime.now(datetime.timezone.utc).astimezone().tzinfo

config = Config()
ciq = CityIq(config, start_time='20180901')

In [4]:
locations = gpd.GeoDataFrame(ciq.locations_dataframe, geometry='geometry')
assets = gpd.GeoDataFrame(ciq.asset_dataframe, geometry='geometry')

locations.head()

,locationUid,locationType,parentLocationUid,geometry
0,004361eb,WALKWAY,004361eb,LINESTRING (-117.1706148251287 32.726548905684...
1,00456472,TRAFFIC_LANE,00456472,LINESTRING (-117.1705245131968 32.726558517207...
2,0051796c,WALKWAY,0051796c,LINESTRING (-117.1666581391174 32.722214661862...
3,00537bf3,TRAFFIC_LANE,00537bf3,LINESTRING (-117.166581746989 32.7222178855665...
4,005f90ed,WALKWAY,005f90ed,LINESTRING (-117.1665182431566 32.723810889902...


In [6]:
len(locations)

6616

In [7]:
assets.head()

,assetUid,assetType,parentAssetUid,mediaType,events,geometry
0,000223ee-a868-474b-abcb-12ff1bad00a3,CAMERA,131d67b7-daaf-4252-9202-5df347a6c2a3,"IMAGE,VIDEO",,POINT (-117.1600173 32.71143062)
1,0002e3bb-5a9a-4083-9a96-fad0d22877b9,MIC,e8beafbf-7836-48c3-bb14-440970eecd6d,AUDIO,,POINT (-117.1493049 32.70997207)
2,0003806e-e409-449f-aab1-fc95f6e88d4e,CAMERA,f91ec557-f03e-4189-8b83-5aa8d28fbf14,"IMAGE,VIDEO",,POINT (-117.1566029 32.70934863)
3,000a4ac7-224b-4fb7-a2e2-cd0592d2b2de,CAMERA,741ce488-72fa-4216-a65d-216f7bdf8b7d,"IMAGE,VIDEO",,POINT (-117.148666 32.71585673)
4,000b2365-5309-422a-9be6-1b7127ca18db,NODE,None,None,,POINT (-117.264282290807 32.8105285177251)


In [8]:
locations.locationType.value_counts()

WALKWAY         2722
TRAFFIC_LANE    2546
PARKING_ZONE    1348
Name: locationType, dtype: int64

In [9]:
assets.assetType.value_counts()

CAMERA        12360
MIC            5615
EM_SENSOR      3184
NODE           2808
ENV_SENSOR     2807
Name: assetType, dtype: int64

In [10]:
!pwd


/Users/eric/proj/virt-proj/data-project/sdrdl-data-projects/sandiego.gov/sandiego.gov-cityiq_assets/notebooks
